# Visualize the results of each model in aggregate

In [2]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from plotly.graph_objs import *

In [3]:
import pandas as pd
import numpy as np

In [24]:
from wisdm import wisdm
wisdm.set_data(version="2")

# Active Learning with Uncertainty Sampling By Threshold

In [45]:
results = []
bad_user_ids = []
for user_id in wisdm.user_ids:
    try:
        user_results_df = pd.read_pickle("./results/experiment_08-29_train_v1_test_v2_active1/"+user_id+".pickle")
        results.append(user_results_df)
    except FileNotFoundError as fnfe:
        print("%s not found : user may not have had enough labeled data" % user_id)
        bad_user_ids.append(user_id)
        pass
        
results_df = pd.concat(results).reset_index(drop=True)


1269 not found : user may not have had enough labeled data
1276 not found : user may not have had enough labeled data
1480 not found : user may not have had enough labeled data
1511 not found : user may not have had enough labeled data
1679 not found : user may not have had enough labeled data
1724 not found : user may not have had enough labeled data
1750 not found : user may not have had enough labeled data


In [46]:
results_df.describe()

,impersonal score Mean,impersonal score STD,k-run,personal + cluster score Mean,personal + cluster score STD,personal + impersonal score Mean,personal + impersonal score STD,personal score Mean,personal score STD,personal training data,uncertainty threshold
count,4480.000000,4480.0,4480.000000,3981.000000,3981.000000,3981.000000,3.981000e+03,3981.000000,3981.000000,4480.000000,3380.000000
mean,0.375530,0.0,4.500000,0.826434,0.077707,0.830670,7.600626e-02,0.797763,0.090806,71.853795,0.500000
std,0.344764,0.0,2.872602,0.199797,0.099323,0.200214,9.877614e-02,0.221443,0.113927,91.135048,0.187111
min,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.200000
25%,0.000000,0.0,2.000000,0.716667,0.000000,0.729167,0.000000e+00,0.654762,0.000000,10.000000,0.350000
50%,0.304348,0.0,4.500000,0.888158,0.012559,0.895833,1.110223e-16,0.855219,0.000000,50.000000,0.500000
75%,0.611111,0.0,7.000000,1.000000,0.157135,1.000000,1.530931e-01,1.000000,0.192847,89.000000,0.650000
max,1.000000,0.0,9.000000,1.000000,0.500000,1.000000,5.000000e-01,1.000000,0.443412,509.000000,0.800000


## Graph Uncertainty Threshold by Amt. of Personal Training Data by Personal + Impersonal Accuracy

In [47]:
uncertainty_thresholds = np.arange(0.2, 0.8, 0.05)

personal_means = {}
personal_plus_universal_means = {}
personal_plus_cluster_means = {}
universal_means = results_df['impersonal score Mean']

for thresh in uncertainty_thresholds:
    personal_scores = results_df[results_df['uncertainty threshold'] == thresh]['personal score Mean']
    personal_means[ts] = personal_scores
    
    personal_plus_universal_scores = results_df[results_df['uncertainty threshold'] == thresh]['personal + impersonal score Mean']
    personal_plus_universal_means[ts] = personal_plus_universal_scores
    
    personal_plus_cluster_scores = results_df[results_df['uncertainty threshold'] == thresh]['personal + cluster score Mean']
    personal_plus_cluster_means[ts] = personal_plus_cluster_scores

In [48]:
graph_df = results_df[results_df['uncertainty threshold'] == thresh]
graph_df.head()

,classifier,impersonal score Mean,impersonal score STD,k-run,personal + cluster score Mean,personal + cluster score STD,personal + impersonal score Mean,personal + impersonal score STD,personal score Mean,personal score STD,personal training data,test user,uncertainty threshold
12,RF with Wiki Parameters,0.631579,0.0,0,0.842105,0.209327,0.923445,0.110774,0.822967,0.239903,149,194,0.8
25,RF with Wiki Parameters,0.555556,0.0,1,0.878788,0.168871,0.904040,0.129946,0.838384,0.221532,150,194,0.8
38,RF with Wiki Parameters,0.611111,0.0,2,0.858586,0.188567,0.919192,0.114503,0.838384,0.221532,150,194,0.8
51,RF with Wiki Parameters,0.588235,0.0,3,0.871658,0.173447,0.877005,0.163430,0.828877,0.230505,151,194,0.8
64,RF with Wiki Parameters,0.647059,0.0,4,0.850267,0.198078,0.887701,0.155541,0.834225,0.224089,151,194,0.8


In [44]:
import colorlover as cl
from IPython.display import HTML

In [67]:
#trace = Scatter(
#            y=results_df[results_df['personal training data'],
#            x=results_df['uncertainty threshold'],
#            mode='markers',
#            marker=dict(color=['red' if acc > 0.9 else 'blue' for acc in results_df['personal + impersonal score Mean']], 
#                        #colorscale='Jet',
#                        opacity=0.2))

good_trace = Scatter(
                y=results_df[results_df['personal + impersonal score Mean'] > 0.9]['personal training data'],
                x=results_df[results_df['personal + impersonal score Mean'] > 0.9]['uncertainty threshold'],
                mode='markers',
                marker=dict(color='red', opacity=0.5),
                name='Accuracy > 0.9')

bad_trace = Scatter(
                y=results_df[results_df['personal + impersonal score Mean'] <= 0.9]['personal training data'],
                x=results_df[results_df['personal + impersonal score Mean'] <= 0.9]['uncertainty threshold'],
                mode='markers',
                marker=dict(color='blue', opacity=0.5),
                name='Accuracy <= 0.9')
data = [good_trace, bad_trace]
layout=Layout(title="Uncertainty Threshold vs Personal Training Data vs. Accuracy",
              yaxis=dict(title='# of personal samples in training set'),
              xaxis=dict(title='Difference between maximum class probability and uniform probability when defining certainty')
              #paper_bgcolor='rgba(0,0,255,255)',
              #plot_bgcolor='rgba(0,0,255,255)'
             )
fig = Figure(data=data, layout=layout)
iplot(fig)

## Box Plot Personal Training Amount Bins by accuracy

In [71]:
training_sizes = [10,20,30,40,50,60,70,80,90,100]
training_bins = [(0,0.1)]
training_bins += [(ts-10, ts) for ts in training_sizes]

personal_means = {}
personal_plus_universal_means = {}
personal_plus_cluster_means = {}
universal_means = results_df['impersonal score Mean']

for ts in training_bins:
    personal_scores = results_df[(results_df['personal training data'] >= ts[0]) & \
                                 (results_df['personal training data'] < ts[1])]['personal score Mean']
    personal_means[ts[1]] = personal_scores
    
    personal_plus_universal_scores = results_df[(results_df['personal training data'] >= ts[0]) & \
                                                (results_df['personal training data'] < ts[1])]['personal + impersonal score Mean']
    personal_plus_universal_means[ts[1]] = personal_plus_universal_scores
    
    personal_plus_cluster_scores = results_df[(results_df['personal training data'] >= ts[0]) & \
                                              (results_df['personal training data'] < ts[1])]['personal + cluster score Mean']
    personal_plus_cluster_means[ts[1]] = personal_plus_cluster_scores

In [78]:
universal_trace = Box(y=universal_means,
                      x=[0]*len(universal_means),
                      name="universal model",
                      )

data = [universal_trace]

personal_x = []
personal_plus_universal_x = []
personal_plus_cluster_x = []

personal_means_all = []
personal_plus_universal_means_all = []
personal_plus_cluster_means_all = []

for ts in training_sizes:
    personal_means_all += personal_means[ts].tolist()
    personal_x += [ts] * len(personal_means[ts])
    
    personal_plus_universal_means_all += personal_plus_universal_means[ts].tolist()
    personal_plus_universal_x += [ts] *len(personal_plus_universal_means[ts])
    
    personal_plus_cluster_means_all += personal_plus_cluster_means[ts].tolist()
    personal_plus_cluster_x += [ts] * len(personal_plus_cluster_means[ts])

personal_trace = Box(y=personal_means_all,
                     x=personal_x,
                     boxpoints='all',
                     jitter=0.8,
                     marker=dict(opacity=0.3),
                     name="Personal")

data.append(personal_trace)

personal_plus_universal_trace = Box(y=personal_plus_universal_means_all,
                                    x=personal_plus_universal_x,
                                    boxpoints='all',
                                    jitter=0.8,
                                    marker=dict(opacity=0.3),
                                    name="Personal + Universal")

data.append(personal_plus_universal_trace)

personal_plus_cluster_trace = Box(y=personal_plus_cluster_means_all,
                                    x=personal_plus_cluster_x,
                                    boxpoints='all',
                                    jitter=0.8,
                                  marker=dict(opacity=0.3),
                                    name="Personal + Cluster")

data.append(personal_plus_cluster_trace)

layout = Layout(showlegend=True,
                title="Unstratified WISDM V2 w/v1.1 for impersonal data : Performance of model personalization with varying personal training data",
                boxmode='group',
                yaxis=dict(title="Accuracy in %"),
                xaxis=dict(title="Amount of training data or training burden to the user"))
fig = Figure(data=data, layout=layout)

iplot(fig, filename="boxplots_of_wisdm_models_uncertainty_sampling_by_threshold")

In [73]:
len(personal_means_all)

3565

# Active Learning By Least Certain Sampling

In [34]:
training_sizes = [10,20,30,40,50,60,70,80,90,100]

personal_means = {}
personal_plus_universal_means = {}
personal_plus_cluster_means = {}
universal_means = results_df['impersonal score Mean']

for ts in training_sizes:
    personal_scores = results_df[results_df['personal training data'] == ts]['personal score Mean']
    personal_means[ts] = personal_scores
    
    personal_plus_universal_scores = results_df[results_df['personal training data'] == ts]['personal + impersonal score Mean']
    personal_plus_universal_means[ts] = personal_plus_universal_scores
    
    personal_plus_cluster_scores = results_df[results_df['personal training data'] == ts]['personal + cluster score Mean']
    personal_plus_cluster_means[ts] = personal_plus_cluster_scores

In [35]:
universal_trace = Box(y=universal_means,
                      x=[0]*len(universal_means),
                      name="universal model",
                      )

data = [universal_trace]

personal_x = []
personal_plus_universal_x = []
personal_plus_cluster_x = []

personal_means_all = []
personal_plus_universal_means_all = []
personal_plus_cluster_means_all = []

for ts in training_sizes:
    personal_means_all += personal_means[ts].tolist()
    personal_x += [ts] * len(personal_means[ts])
    
    personal_plus_universal_means_all += personal_plus_universal_means[ts].tolist()
    personal_plus_universal_x += [ts] *len(personal_plus_universal_means[ts])
    
    personal_plus_cluster_means_all += personal_plus_cluster_means[ts].tolist()
    personal_plus_cluster_x += [ts] * len(personal_plus_cluster_means[ts])

personal_trace = Box(y=personal_means_all,
                     x=personal_x,
                     name="Personal")

data.append(personal_trace)

personal_plus_universal_trace = Box(y=personal_plus_universal_means_all,
                                    x=personal_plus_universal_x,
                                    name="Personal + Universal")

data.append(personal_plus_universal_trace)

personal_plus_cluster_trace = Box(y=personal_plus_cluster_means_all,
                                    x=personal_plus_cluster_x,
                                    name="Personal + Cluster")

data.append(personal_plus_cluster_trace)

layout = Layout(showlegend=True,
                title="Unstratified WISDM V2 w/v1.1 for impersonal data : Performance of model personalization with varying personal training data",
                boxmode='group',
                yaxis=dict(title="Accuracy in %"),
                xaxis=dict(title="Amount of training data or training burden to the user"))
fig = Figure(data=data, layout=layout)

iplot(fig, filename="boxplots_of_wisdm_models")

# By User

In [113]:
def plotScoresByUser(user_id):
    personal_means = {}
    personal_plus_universal_means = {}
    personal_plus_cluster_means = {}
    universal_means = results_df['impersonal score Mean']

    for ts in training_sizes:
        personal_scores = results_df[(results_df['personal training data'] == ts) & \
                                     (results_df['test user'] == user_id)]['personal score Mean']
        personal_means[ts] = personal_scores

        personal_plus_universal_scores = results_df[(results_df['personal training data'] == ts) & \
                                     (results_df['test user'] == user_id)]['personal + impersonal score Mean']
        personal_plus_universal_means[ts] = personal_plus_universal_scores

        personal_plus_cluster_scores = results_df[(results_df['personal training data'] == ts) & \
                                     (results_df['test user'] == user_id)]['personal + cluster score Mean']
        personal_plus_cluster_means[ts] = personal_plus_cluster_scores
    
    
    universal_trace = Box(y=universal_means,
                      x=[0]*len(universal_means),
                      name="universal model",
                      boxpoints='suspectedoutliers',
                      )

    data = [universal_trace]

    personal_x = []
    personal_plus_universal_x = []
    personal_plus_cluster_x = []

    personal_means_all = []
    personal_plus_universal_means_all = []
    personal_plus_cluster_means_all = []

    for ts in training_sizes:
        personal_means_all += personal_means[ts].tolist()
        personal_x += [ts] * len(personal_means[ts])

        personal_plus_universal_means_all += personal_plus_universal_means[ts].tolist()
        personal_plus_universal_x += [ts] *len(personal_plus_universal_means[ts])

        personal_plus_cluster_means_all += personal_plus_cluster_means[ts].tolist()
        personal_plus_cluster_x += [ts] * len(personal_plus_cluster_means[ts])

    personal_trace = Box(y=personal_means_all,
                         x=personal_x,
                         name="Personal",
                         boxpoints="suspectedoutliers")

    data.append(personal_trace)

    personal_plus_universal_trace = Box(y=personal_plus_universal_means_all,
                                        x=personal_plus_universal_x,
                                        name="Personal + Universal",
                                        boxpoints="suspectedoutliers")

    data.append(personal_plus_universal_trace)

    personal_plus_cluster_trace = Box(y=personal_plus_cluster_means_all,
                                        x=personal_plus_cluster_x,
                                        name="Personal + Cluster",
                                        boxpoints="suspectedoutliers")

    data.append(personal_plus_cluster_trace)

    layout = Layout(showlegend=True, boxmode='group')
    fig = Figure(data=data, layout=layout)

    iplot(fig, filename="boxplots_of_wisdm_models")

In [114]:
plotScoresByUser(wisdm.user_ids[0])

For this first user we observe the following.
* the personal+cluster approach almost always does better, on average.
* the personal+cluster appraoch never does as poorly as either the universal+cluster, universal+personal
* the personal+cluster appraoch offers GREAT improvements over other approaches around 20-40 samples

In [115]:
plotScoresByUser(wisdm.user_ids[1])

For the second user, we oddly enough see that personal models and the personal+cluster model both outperform the personal+universal model regardless of the amount of personal data and by a great deal

In [116]:
plotScoresByUser(wisdm.user_ids[2])

Here, we see a different story again.  The personal+universal model not only does well on average, it's worst case scenario is occasionally better than those of the personal or personal+cluster models.

# Select users who get the most from each kind of model

In [117]:
def getModelAccuracyMean(user_id, ts):
    # each list element at index, i, represents that model improvement over the best other model with training size[k]
    user_df = results_df[(results_df['test user'] == user_id) & \
                         (results_df['personal training data'] == ts)]
    personal_score_mean = user_df['personal score Mean'].mean()
    impersonal_score_mean = user_df['impersonal score Mean'].mean()
    personal_plus_impersonal_mean = user_df['personal + impersonal score Mean'].mean()
    personal_plus_cluster_mean = user_df['personal + cluster score Mean'].mean()

    #print("personal : %s" % personal_score_mean)
    #print("impersonal : %s" % impersonal_score_mean)
    #print("personal + impersonal : %s" % personal_plus_impersonal_mean)
    #print("personal + cluster : %s" % personal_plus_cluster_mean)
    mean_scores = {"personal" : personal_score_mean,
                   "impersonal" : impersonal_score_mean,
                   "personal + impersonal" : personal_plus_impersonal_mean,
                   "personal + cluster" : personal_plus_cluster_mean}
    return mean_scores

In [118]:
model_means_columns = ['user id', 'personal', 'impersonal', 'personal + impersonal', 'personal + cluster']
model_means = []

for user_id in wisdm.user_ids:
    if user_id not in bad_user_ids:
        mean_scores = getModelAccuracyMean(user_id, 10)
        mean_scores['user id'] = user_id
        model_means.append(mean_scores)
    
scores_df = pd.DataFrame(model_means, columns=model_means_columns)

In [119]:
scores_df.describe()

,personal,impersonal,personal + impersonal,personal + cluster
count,35.000000,35.000000,35.000000,35.000000
mean,0.897739,0.358388,0.912250,0.908759
std,0.115055,0.307850,0.099780,0.102133
min,0.595778,0.000000,0.600000,0.618444
25%,0.838376,0.081988,0.872833,0.849889
50%,0.921739,0.266667,0.930909,0.932857
75%,1.000000,0.551840,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000


In [120]:
users_benefit_from_personal = []
users_benefit_from_impersonal = []
users_benefit_from_personal_plus_impersonal = []
users_benefit_from_personal_plus_cluster = []

for ind, row in scores_df.iterrows():
    scores = [row['personal'], row['impersonal'], row['personal + impersonal'], row['personal + cluster']]
    best_model = np.argmax(scores)
    
    if best_model == 0:
        users_benefit_from_personal.append(row['user id'])
    elif best_model == 1:
        users_benefit_from_impersonal.append(row['user id'])
    elif best_model == 2:
        users_benefit_from_personal_plus_impersonal.append(row['user id'])
    elif best_model == 3:
        users_benefit_from_personal_plus_cluster.append(row['user id'])

In [121]:
print("Personal was best : %s" % len(users_benefit_from_personal))
print("Impersonal was best : %s" % len(users_benefit_from_impersonal))
print("Personal + Impersonal was best : %s" % len(users_benefit_from_personal_plus_impersonal))
print("Personal + Cluster was best : %s" % len(users_benefit_from_personal_plus_cluster))


Personal was best : 20
Impersonal was best : 1
Personal + Impersonal was best : 10
Personal + Cluster was best : 6


In [122]:
def getBests(training_size):
    model_means_columns = ['user id', 'personal', 'impersonal', 'personal + impersonal', 'personal + cluster']
    model_means = []

    for user_id in wisdm.user_ids:
        if user_id not in bad_user_ids:
            mean_scores = getModelAccuracyMean(user_id, training_size)
            mean_scores['user id'] = user_id
            model_means.append(mean_scores)

    scores_df = pd.DataFrame(model_means, columns=model_means_columns)
    
    users_benefit_from_personal = []
    users_benefit_from_impersonal = []
    users_benefit_from_personal_plus_impersonal = []
    users_benefit_from_personal_plus_cluster = []

    for ind, row in scores_df.iterrows():
        scores = [row['personal'], row['impersonal'], row['personal + impersonal'], row['personal + cluster']]
        best_model = np.argmax(scores)

        if best_model == 0:
            users_benefit_from_personal.append(row['user id'])
        elif best_model == 1:
            users_benefit_from_impersonal.append(row['user id'])
        elif best_model == 2:
            users_benefit_from_personal_plus_impersonal.append(row['user id'])
        elif best_model == 3:
            users_benefit_from_personal_plus_cluster.append(row['user id'])
    return users_benefit_from_personal, users_benefit_from_impersonal, \
            users_benefit_from_personal_plus_impersonal, users_benefit_from_personal_plus_cluster

In [123]:
training_sizes = [10,20,30,40,50,60,70,80,90,100]

personal_bests = []
impersonal_bests = []
personal_impersonal_bests = []
personal_cluster_bests = []

for ts in training_sizes:
    personal, impersonal, personal_impersonal, personal_cluster = getBests(ts)
    
    personal_bests.append(personal)
    impersonal_bests.append(impersonal)
    personal_impersonal_bests.append(personal_impersonal)
    personal_cluster_bests.append(personal_cluster)
    
    print("Training Size : %s" % ts)
    print("\t personal : %s" % len(personal))
    print("\t impersonal : %s" % len(impersonal))
    print("\t personal + impersonal : %s" % len(personal_impersonal))
    print("\t personal + cluster : %s" % len(personal_cluster))

Training Size : 10
	 personal : 20
	 impersonal : 1
	 personal + impersonal : 10
	 personal + cluster : 6
Training Size : 20
	 personal : 25
	 impersonal : 0
	 personal + impersonal : 9
	 personal + cluster : 3
Training Size : 30
	 personal : 23
	 impersonal : 0
	 personal + impersonal : 9
	 personal + cluster : 5
Training Size : 40
	 personal : 27
	 impersonal : 0
	 personal + impersonal : 6
	 personal + cluster : 4
Training Size : 50
	 personal : 28
	 impersonal : 0
	 personal + impersonal : 5
	 personal + cluster : 4
Training Size : 60
	 personal : 26
	 impersonal : 0
	 personal + impersonal : 8
	 personal + cluster : 3
Training Size : 70
	 personal : 27
	 impersonal : 0
	 personal + impersonal : 7
	 personal + cluster : 3
Training Size : 80
	 personal : 29
	 impersonal : 1
	 personal + impersonal : 4
	 personal + cluster : 3
Training Size : 90
	 personal : 30
	 impersonal : 0
	 personal + impersonal : 4
	 personal + cluster : 3
Training Size : 100
	 personal : 30
	 impersonal : 0
	

In [124]:
personal_trace = Scatter(x=training_sizes,
                     y=[len(x) for x in personal_bests],
                     name="Personal")
impersonal_trace = Scatter(x=training_sizes,
                       y=[len(x) for x in impersonal_bests],
                       name="Impersonal")
personal_impersonal_trace = Scatter(x=training_sizes,
                       y=[len(x) for x in personal_impersonal_bests],
                       name="Personal + Impersonal")
personal_cluster_trace = Scatter(x=training_sizes,
                       y=[len(x) for x in personal_cluster_bests],
                       name="Personal + Cluster")

data = [personal_trace, impersonal_trace, personal_impersonal_trace, personal_cluster_trace]
layout=Layout(title="WISDM V2 w/v1.1 training : #users who get the best performance from a model as the personal training set increases",
              yaxis=dict(range=[0,40],
                  title="Number of users who get the best performance from this model"),
              xaxis=dict(title="Amount of personal training data or user training burden"))
fig = Figure(data=data, layout=layout)
iplot(fig, filename="user_model_bests")

In [82]:
import scipy

In [84]:
scipy.stats.mode([1,1,1,1,4,5,6,7]).mode[0]

1